In [6]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ml-dtypes<0.5.0,>=0.3.1 (from tensorflow-intel) (from versions: none)
ERROR: No matching distribution found for ml-dtypes<0.5.0,>=0.3.1



INFO: pip is looking at multiple versions of tensorflow-intel to determine which version is compatible with other requirements. This could take a while.


In [7]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade pip transformers tensorflow

In [2]:
pip install transformers torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

# Sample text corpus
text = "hello world. this is a simple text generation example using pytorch."

# Character mappings
chars = sorted(list(set(text)))
char2idx = {ch: idx for idx, ch in enumerate(chars)}
idx2char = {idx: ch for idx, ch in enumerate(chars)}

# Hyperparameters
seq_length = 10
hidden_size = 128
num_layers = 1
lr = 0.003
num_epochs = 300

# Prepare dataset
def create_sequences(text, seq_length):
    inputs, targets = [], []
    for i in range(len(text) - seq_length):
        inputs.append([char2idx[ch] for ch in text[i:i+seq_length]])
        targets.append(char2idx[text[i+seq_length]])
    return torch.tensor(inputs), torch.tensor(targets)

X, y = create_sequences(text, seq_length)

# Model definition
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h):
        out, h = self.rnn(x, h)
        out = self.fc(out[:, -1, :])  # take output from last time step
        return out, h

model = CharRNN(len(chars), hidden_size, len(chars), num_layers)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# One-hot encoding function
def one_hot_encode(x, dict_size):
    result = torch.zeros(x.size(0), x.size(1), dict_size)
    for i in range(x.size(0)):
        for j in range(x.size(1)):
            result[i][j][x[i][j]] = 1
    return result

# Training loop
for epoch in range(num_epochs):
    h = torch.zeros(num_layers, X.size(0), hidden_size)
    inputs = one_hot_encode(X, len(chars))
    outputs, h = model(inputs, h.detach())
    loss = criterion(outputs, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Text generation
def generate_text(start_seq, length=100):
    model.eval()
    chars_seq = list(start_seq.lower())
    input_seq = torch.tensor([[char2idx[c] for c in chars_seq]])
    input_seq = one_hot_encode(input_seq, len(chars))
    h = torch.zeros(num_layers, 1, hidden_size)

    result = start_seq
    for _ in range(length):
        output, h = model(input_seq, h)
        prob = nn.functional.softmax(output[0], dim=0).data
        char_idx = torch.multinomial(prob, 1)[0]
        char = idx2char[char_idx.item()]
        result += char
        input_seq = one_hot_encode(torch.tensor([[char2idx[c] for c in result[-seq_length:]]]), len(chars))

    return result

# Example generation
print("\nGenerated Text:\n")
print(generate_text("hello worl"))

Epoch [50/300], Loss: 0.0224
Epoch [100/300], Loss: 0.0031
Epoch [150/300], Loss: 0.0019
Epoch [200/300], Loss: 0.0013
Epoch [250/300], Loss: 0.0009
Epoch [300/300], Loss: 0.0007

Generated Text:

hello worldxstx tg ea etx t xst st xml ea eta eixetxstgssaxst xt  en mdei eag st staxt xmtxxtihsl et et htxhta


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Sample corpus
corpus = """
deep learning is a subset of machine learning.
machine learning is a part of artificial intelligence.
neural networks are used in deep learning.
artificial intelligence is transforming the world.
data science involves statistics and machine learning.
"""

# Preprocessing
corpus = corpus.lower().split()
vocab = sorted(set(corpus))
word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for w, i in word2idx.items()}

# Hyperparameters
seq_length = 4
hidden_size = 128
num_layers = 1
lr = 0.01
num_epochs = 300

# Create sequences
def create_sequences(words, seq_len):
    X, y = [], []
    for i in range(len(words) - seq_len):
        X.append([word2idx[w] for w in words[i:i+seq_len]])
        y.append(word2idx[words[i+seq_len]])
    return torch.tensor(X), torch.tensor(y)

X, y = create_sequences(corpus, seq_length)

# Model
class WordLSTM(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, 50)
        self.lstm = nn.LSTM(50, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

model = WordLSTM(len(vocab), hidden_size, num_layers)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training
for epoch in range(num_epochs):
    hidden = (torch.zeros(num_layers, X.size(0), hidden_size),
              torch.zeros(num_layers, X.size(0), hidden_size))

    output, hidden = model(X, hidden)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Text generation
def generate_text(start_words, gen_len=20):
    model.eval()
    words = start_words.lower().split()
    state_h = torch.zeros(num_layers, 1, hidden_size)
    state_c = torch.zeros(num_layers, 1, hidden_size)

    for _ in range(gen_len):
        x = torch.tensor([[word2idx.get(w, 0) for w in words[-seq_length:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_idx = np.random.choice(len(last_word_logits), p=p)

        words.append(idx2word[word_idx])

    return ' '.join(words)

# Example
print("\nGenerated Completion:\n")
print(generate_text("deep learning is"))


Epoch [50/300], Loss: 0.0002
Epoch [100/300], Loss: 0.0001
Epoch [150/300], Loss: 0.0001
Epoch [200/300], Loss: 0.0001
Epoch [250/300], Loss: 0.0000
Epoch [300/300], Loss: 0.0000

Generated Completion:

deep learning is subset of machine learning. machine learning. machine learning is a part machine learning. machine learning. machine learning is a part


In [2]:
import openai

# Set your OpenAI API key here
openai_client = openai.OpenAI(api_key="   ......")

# Function to generate text with tone control using GPT
def generate_text_with_gpt(base_content, tone="formal", model="gpt-3.5-turbo"):
    # Step 1: Create a system prompt
    system_prompt = f"You are an expert content writer. Write the text in a {tone} tone."

    # Step 2: Create user prompt
    user_prompt = f"Base content: {base_content}\n\nGenerate the final text accordingly."

    # Step 3: Call OpenAI GPT API (New Format)
    response = openai_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )

    # Step 4: Extract and return the generated content
    final_text = response.choices[0].message.content.strip()
    return final_text

# Example usage
if __name__ == "__main__":
    # Input content
    base_content = input('Enter the text:- ')

    # Control settings
    tone = input("Enter the tone type to print:-")  # Try formal, casual, friendly, emotional, professional, etc.

    # Generate controlled text using GPT
    result = generate_text_with_gpt(base_content, tone)
    print("Generated Text:\n")
    print(result)


Enter the text:-  wakeup early because
Enter the tone type to print:- formal


Generated Text:

It is highly recommended to wake up early because it can have a significant impact on your productivity, mood, and overall well-being. By rising early, you can seize the day and make the most out of your time. Embracing the stillness of the early morning hours allows for increased focus and concentration, enabling you to tackle tasks with a clear mind and heightened energy levels. Additionally, waking up early provides the opportunity to establish a healthy routine, including exercise, proper nutrition, and self-care practices - all of which are essential for a balanced and fulfilling lifestyle. Therefore, setting the habit of waking up early can lead to improved efficiency, enhanced mental clarity, and a greater sense of accomplishment throughout the day.


In [1]:
# create Gui text generator

# Module import
import tkinter as tk
from PIL import Image, ImageTk  # Pillow library
import tkinter as tk
from tkinter import messagebox
# import mysql.connector
# from mysql.connector import Error
import openai



# Create GUI
root = tk.Tk()
root.title("Text Generator with Tone")
root.geometry("900x700")
root.configure(bg='black')



openai_client=openai.OpenAI(api_key=".....")

def generate_text_with_gpt(base_content, tone="formal", model="gpt-3.5-turbo"):
    # Api key insert 
    # openai_client=openai.OpenAI(api_key="......")
    
    # Step 1: Create a system prompt
    system_prompt = f"You are an expert content writer. Write the text in a {tone} tone."

    # Step 2: Create user prompt
    user_prompt = f"Base content: {base_content}\n\nGenerate the final text accordingly."

    # Step 3: Call OpenAI GPT API (New Format)
    response = openai_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )

    # Step 4: Extract and return the generated content
    final_text = response.choices[0].message.content.strip()
    return final_text

def text():
    openai_client=openai.OpenAI(api_key=",......")
    
    k=Text_entry.get()
    k2=Tone_entry.get()
    result = generate_text_with_gpt(k,k2)
    # print(result)
    # generated_output=result
    # print(generated_output)
    generated_output.insert(tk.END,result)
    
    


# Text Label & Entry
tk.Label(root, text="Enter Text:",font=("Arial", 15),bg="lightblue").grid(row=0, column=0, padx=10,pady=10)
Text_entry = tk.Entry(root, font=("Arial", 15), width=50)
Text_entry.grid(row=0, column=1, padx=10, pady=10)

# Tone Label & Entry
tk.Label(root, text="Tone Type:",font=("Arial", 15),bg="lightblue").grid(row=1,column=0, padx=10, pady=10)
Tone_entry = tk.Entry(root, font=("Arial", 15), width=50)
Tone_entry.grid(row=1, column=1, padx=10, pady=10)

# Generate Button
tk.Button(root, text="Generate Text", command=text, font=("Arial", 15), bg="lightgreen").grid(row=2, columnspan=2, pady=20)

# Output Label & Text Area
tk.Label(root, text="Generated Output:",font=("Arial", 15),bg="lime").grid(row=3,column=0,padx=10,pady=10)
generated_output = tk.Text(root,font=("Arial", 14),width=70,height=10,wrap=tk.WORD, bg="lightyellow")
generated_output.grid(row=4, columnspan=2, padx=20, pady=10)

root.mainloop()
